In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset
tokenizer = LlamaTokenizer.from_pretrained("/data/zhangyi/hf_model/Llama2-7b-chat-hf")
model  = LlamaForCausalLM.from_pretrained("/data/zhangyi/hf_model/Llama2-7b-chat-hf",
                                           
                                          
                                          )
data_files = {
    "train" : "/data/zhangyi/dpo/dpo_other/data/prompt_train_final.json",
    "test": "/data/zhangyi/dpo/dpo_other/data/prompt_test_final.json",
    "valid": "/data/zhangyi/dpo/dpo_other/data/prompt_val_final.json"
}
dataset = load_dataset('json', data_files=data_files)
dataset_train  = dataset['train']
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

FileNotFoundError: Unable to find '/data/zhangyi/dpo/dpo_other/data/prompt_train_final.json'

In [ ]:
import torch
# from Constructor import TrainPrompt,Movie,Evaluate_prompt
import transformers
from typing import List
from datasets import load_dataset
import json
from transformers import AutoModelForCausalLM, AutoTokenizer,GenerationConfig
import torch.nn as nn
from torch.utils.data import DataLoader
import random

device_map = "auto"
def main(
    base_model: str = "",
    lora_weights: str = "",
    test_data_path: dict =  {
    "train": "./data/train.json",
    "validation": "./data/validation.json",
    "test": "./data/test.json"
},
    batch_size : int = 8,
):
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='decapoda-research/llama-7b-hf'"
    
    tokenizer = AutoTokenizer.from_pretrained(base_model) 
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        # load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )
    if lora_weights != "":
        model.load_adapter(lora_weights)
    
    tokenizer.padding_side ="left"
    tokenizer.pad_token_id = (0)

    model.eval()

    def output_generate(
        prompts,
        temperature = 0,
        
        
    ):
        inputs = tokenizer(prompts,return_tensors="pt",truncation=True,padding=True,max_length=1024).to(model.device)
        generation_config = GenerationConfig(
            temperature = temperature,
        )
        generation_output = model.generate(
            **inputs,
            generation_config = generation_config,
            return_dict_in_generate = True,
            output_scores = True,
            max_new_tokens = 1024
        )
        s = generation_output.sequences
        output = tokenizer.batch_decode(s,skip_special_tokens=True)
        output = [_.strip() for _ in output]
        
        return output
    
    
    def convert_dict_to_prompt(d:dict):
        t = Evaluate_prompt()
        t.instructionPrompt = d['instructionPrompt']
        t.gender = d["gender"]
        t.age = d["age"]
        t.occupation = d["occupation"]
        t.historyList = list(map(lambda o:Movie(o["id"],o["title"],o["genre"]),d["historyList"]))
        t.itemList = list(map(lambda o:Movie(o["id"],o["title"],o["genre"]),d["itemList"]))
        t.trueSelection =list(map(lambda o:Movie(o["id"],o["title"],o["genre"]),d["trueSelection"]))
        return t
    def tokenize(prompt,add_eos_token=True):
        result = tokenizer(
            prompt,
            truncation=True,
            max_length = 1024,
            padding = False,
            return_tensors = "pt",
        )
        return result   
    def get_true_selection(data_point):
        true_selection = data_point["trueSelection"]
        titles =[movie['title'] for movie in true_selection]
        return titles
    def generate_and_tokenized_prompt(data_point):
        prompt = str(convert_dict_to_prompt(data_point))
        return {'prompt':prompt,"true_selection":get_true_selection(data_point)}


    class MetricBase:
        def __init__(self):
            raise NotImplementedError()
        def update(self,y_truth,y_pred):
            raise NotImplementedError()
        def get_metric():
            raise NotImplementedError()
        def get_last(self):
            raise NotImplementedError()
    class MetricF1(MetricBase):
        def __init__(self):
            self.sum_TP = 0
            self.sum_FN = 0
            self.sum_FP = 0 
            self.last_TP = 0
            self.last_FN = 0
            self.last_FP = 0
            self.f1 = 0
            self.nums = 0
        def update(self,y_truth:set, y_pred:set):
            self.last_TP = len(y_truth & y_pred)
            self.last_FN = len(y_truth - y_pred)
            self.last_FP = len(y_pred - y_truth)
            self.sum_TP += self.last_TP
            self.sum_FN += self.last_FN
            self.sum_FP += self.last_FP
            self.nums += 1
        def get_metric(self):
            TP = self.sum_TP
            FN = self.sum_FN
            FP = self.sum_FP
            if TP + FN == 0:
                recall = 0
            else:
                recall = TP / (TP + FN)
            if TP + FP == 0:
                precision = 0
            else:
                precision = TP / (TP + FP)
            if precision + recall == 0:
                f1 = 0
            else:
                f1 = 2 * precision * recall / (precision + recall)
            self.recall = recall
            self.precision = precision
            self.f1 = f1
            return f1
        def get_detail(self):
            if not hasattr(self,"recall"):
                f1 = self.get_metric()
            return f1,self.recall,self.precision
        def get_last(self):
            return self.last_TP,self.last_FN,self.last_FP

    def F1_selection(output_text:str,true_selection):
        instruct_text,in_contest = output_text.split("Input: The movie list is as following: ")
        in_contest = in_contest.strip()
        ground_selection,prev_selection = in_contest.split("Output: ")
        ALL_selection = ground_selection.strip().split("\n")
        ALL_selection_set = set(ALL_selection)
        prev_selection = prev_selection.strip().split("\n")
        # true_selection = ALL_selection[:4]
        prev_selection_set = set(prev_selection)
        true_selection_set = set(true_selection)
        return true_selection_set,prev_selection_set

    # test_data = load_dataset("json", data_files=test_data_path)["test"]
    from tqdm import tqdm
    # with open(test_data_path,"r") as f:
    def batch(list,batch_size = batch_size):
        chunk_size = (len(list)-1)//batch_size +1
        for i in range(chunk_size):
            yield list[batch_size * i:batch_size * (i+1)]
    F1_caculate = MetricF1()
    test_data_prompt = []
    test_data_true_selection = []
    # random.shuffle(test_data)
    test_data_load = load_dataset("json", data_files=test_data_path)
    test_datas = test_data_load["test"].map(generate_and_tokenized_prompt)
    test_datas_0 = test_datas[0]
    for test_data in test_datas:
        test_data_prompt.append(test_data['prompt'])

        test_data_true_selection.append(test_data['true_selection'])
    for i,batch in tqdm(enumerate(zip(batch(test_data_prompt),batch(test_data_true_selection)))):
        batch_tokenized_prompt,batch_true_selections = batch
        outputs=output_generate(batch_tokenized_prompt)
        for true_selection,pred_selction in zip(batch_true_selections,outputs):
            true_selection_set,pred_selction_set = F1_selection(pred_selction,true_selection)
            F1_caculate.update(true_selection_set,pred_selction_set)
            F1_caculate.get_metric()
        print(F1_caculate.f1,F1_caculate.recall,F1_caculate.precision,F1_caculate.nums)
    # test_data = test_data.map(generate_and_tokenized_prompt)

        # for i,batch in tqdm(enumerate())
        


if __name__ == "__main__":
    main(
        base_model="lmsys/vicuna-7b-v1.5",
        lora_weights="/storage_fast/yxchen/zhangyi/vicuna-alpaca",
        batch_size=16
        )